# Smart Real Estate Assistant with Amazon Nova

## Workshop Overview
Build an intelligent real estate assistant that combines multiple AI agents to analyze property images, generate marketing content, and provide comprehensive property assessments. This workshop demonstrates Nova's multimodal capabilities in a real-world scenario.

### What You'll Build:
- **Property Analyzer Agent**: Analyzes property images for features, condition, and value indicators
- **Marketing Agent**: Creates compelling property descriptions and marketing materials
- **Valuation Agent**: Provides market insights and pricing recommendations
- **Visual Enhancement Agent**: Generates improved property images for listings

### Real-World Use Case:
A real estate agent uploads property photos and gets instant analysis, marketing copy, valuation insights, and enhanced visuals for listings.

## Setup and Dependencies

In [1]:
# Install required packages
!pip install strands-agents strands-agents-tools  boto3 pillow

In [2]:
from strands import Agent, tool
from strands_tools import generate_image, image_reader
from strands.models import BedrockModel
import json
import os
from datetime import datetime

In [3]:
# Initialize Nova model
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-2-lite-omni-v1:0",
    region_name="us-west-2"
)

## Custom Tools for Real Estate Analysis

In [4]:
@tool
def save_analysis_report(content: str, filename: str) -> str:
    """Save analysis report to file"""
    os.makedirs("output/reports", exist_ok=True)
    filepath = f"output/reports/{filename}"
    with open(filepath, 'w') as f:
        f.write(content)
    return f"Report saved to {filepath}"

@tool
def get_market_data(location: str) -> str:
    """Simulate market data retrieval"""
    # In real implementation, this would connect to MLS or market APIs
    market_data = {
        "average_price_per_sqft": "$250-350",
        "market_trend": "Rising 5% YoY",
        "days_on_market": "25-35 days",
        "comparable_sales": "3 recent sales in area"
    }
    return json.dumps(market_data)

## Agent 1: Property Analyzer
Analyzes property images to identify features, condition, and potential issues.

In [5]:
property_analyzer = Agent(
    model=bedrock_model,
    tools=[image_reader, save_analysis_report],
    system_prompt="""
    You are an expert property analyst. Analyze property images to identify:
    - Room types and layout
    - Condition and maintenance needs
    - Notable features and amenities
    - Potential issues or concerns
    - Estimated square footage and room count
    
    Provide detailed, professional analysis in JSON format with sections:
    - property_overview
    - room_analysis
    - condition_assessment
    - notable_features
    - recommendations
    """
)

## Agent 2: Marketing Content Creator
Creates compelling property descriptions and marketing materials.

In [6]:
marketing_agent = Agent(
    model=bedrock_model,
    tools=[save_analysis_report],
    system_prompt="""
    You are a real estate marketing expert. Create compelling marketing content including:
    - Engaging property description (150-200 words)
    - Key selling points (bullet format)
    - Social media captions
    - Email marketing copy
    
    Focus on emotional appeal while highlighting practical benefits.
    Use active voice and compelling adjectives.
    """
)

## Agent 3: Market Valuation Specialist
Provides pricing insights and market analysis.

In [7]:
valuation_agent = Agent(
    model=bedrock_model,
    tools=[get_market_data, save_analysis_report],
    system_prompt="""
    You are a real estate valuation expert. Analyze property data and market conditions to provide:
    - Estimated market value range
    - Pricing strategy recommendations
    - Market positioning advice
    - Competitive analysis
    
    Consider property condition, features, and local market data.
    Provide conservative and optimistic valuations.
    """
)

## Agent 4: Visual Enhancement Specialist
Generates enhanced property images for marketing.

In [17]:
visual_enhancer = Agent(
    model=bedrock_model,
    tools=[generate_image],
    system_prompt="""
    You are a aws solution architect. Generate enhanced architecture images that:
    - showcase aws best practices
    - Use optimal clarity in each component
    - align proper aws services icons
    - all the texts are clear to show the each component
    - image should align aws architecture
    - FINAL DECISION: <path to final decision image>
    
    """
)

## Demo: Complete Property Analysis Workflow

Let's demonstrate the complete workflow with a sample property.

In [9]:
# Sample property information
property_info = {
    "address": "123 Maple Street, Suburbia",
    "type": "Single Family Home",
    "bedrooms": 3,
    "bathrooms": 2,
    "sqft": 1800,
    "lot_size": "0.25 acres",
    "year_built": 1995
}

print("Property Information:")
print(json.dumps(property_info, indent=2))

Property Information:
{
  "address": "123 Maple Street, Suburbia",
  "type": "Single Family Home",
  "bedrooms": 3,
  "bathrooms": 2,
  "sqft": 1800,
  "lot_size": "0.25 acres",
  "year_built": 1995
}


### Step 1: Generate Sample Property Images

In [18]:
# Generate sample property images for analysis
image_prompt = f"""
Create an AWS architecture diagram following official AWS Well-Architected Framework standards:

INFRASTRUCTURE LAYOUT:
Region: us-west-2
[AWS Cloud Border - Standard AWS light grey border]

VPC (10.0.0.0/16) [Standard AWS VPC icon]
├── AZ: us-west-2a
│   └── Private Subnet (10.0.1.0/24)
├── AZ: us-west-2b
│   └── Private Subnet (10.0.2.0/24)

COMPONENT PLACEMENT (Use official AWS icons):
Edge Tier:

Amazon CloudFront [Use AWS Edge icon]
Custom domain: app.example.com
Network Tier:

Application Load Balancer [Use AWS ALB icon]
Target Groups connected to services
Compute Tier (EKS):

EKS Control Plane [Use AWS EKS icon]
Node Groups in private subnets
Pods visualization (use container icons):
Product Service (x3)
Order Service (x2)
Data Tier:

DynamoDB Tables [Use AWS DynamoDB icon]
product-catalog
order-history
Security Components:

Security Groups [Use AWS SG shield icon]
IAM Roles [Use AWS IAM icon]
VPC Endpoints [Use AWS Endpoint icon]
CONNECTIVITY RULES:
Use AWS standard arrow types:

Solid lines: Direct connections
Dashed lines: Security group references
Orange (#FF9900): Data flow
Blue (#232F3E): Network flow
Standard AWS icon sizes:

Large: Primary services (ALB, EKS)
Medium: Supporting services
Small: Components
LAYOUT STANDARDS:
Left-to-right traffic flow
Services aligned horizontally within tiers
Clear separation between tiers
Consistent spacing
AWS service names below icons
Resource names in standard AWS font
COLOR SCHEME:
AWS Orange: #FF9900
AWS Dark: #232F3E
VPC Blue: #147EBA
Subnet Grey: #E6E6E6"
Key Improvements:

Use only official AWS architectural icons
Maintain consistent icon sizing
Follow AWS networking flow patterns
Include proper security boundaries
Show clear service connections
Include resource naming conventions
"""

print("Generating sample property images...")
sample_images = visual_enhancer(image_prompt)
print(f"Generated images: {sample_images}")

Generating sample property images...

Tool #1: generate_image
Final Decision: output/create_an_aws_architecture_diagram.pngGenerated images: Final Decision: output/create_an_aws_architecture_diagram.png



### Step 2: Analyze Property Images

In [11]:
# Analyze the generated property images
analysis_prompt = f"""
Analyze these property images for: {property_info['address']}
Property details: {json.dumps(property_info)}

Image paths: {sample_images}

Provide comprehensive analysis and save as 'property_analysis.json'
"""

print("Analyzing property images...")
analysis_result = property_analyzer(analysis_prompt)
print("Analysis completed:")
print(analysis_result)

Analyzing property images...

Tool #1: image_reader

Tool #2: save_analysis_report
## Property Analysis Report for 123 Maple Street, Suburbia

### Property Overview
123 Maple Street, Suburbia
**Property Type:** Single Family Home
**Year Built:** 1995
**Lot Size:** 0.25 acres
**Provided Details:**
: 3edrooms
- Bathrooms: 2
- Square Footage: 1,800 sqft

### Room Analysis

#### Living Room
 featuring: image shows a modern, well-lit living room area
- **Flooring:** Hardwood floors in excellent condition
- **Windows:** Large multi-paned windows offering a view of greenery outside
- **Layout:** Open and spacious design
Features:**
  - Large L-shaped sectional sofa with blue and neutral accent pillows
  - Center coffee table with ottoman
  - Ceiling fan with integrated lighting
 large abstract pieces)k (two
  - Side tables with table lamps
  - Indoor plants for decorative appeal
  - Area rug under seating arrangement
  - Grey curtains on windows

 400-500 square feet visible in this image

##

### Step 3: Create Marketing Content

In [12]:
# Generate marketing content based on analysis
marketing_prompt = f"""
Create marketing content for: {property_info['address']}
Property details: {json.dumps(property_info)}
Analysis results: {analysis_result}

Generate comprehensive marketing materials and save as 'marketing_content.txt'
"""

print("Creating marketing content...")
marketing_result = marketing_agent(marketing_prompt)
print("Marketing content created:")
print(marketing_result)

Creating marketing content...

Tool #1: save_analysis_report
Great! I've created comprehensive marketing content for 123 Maple Street, Suburbia, and saved it to **marketing_content.txt**. The content includes:

1. **Engaging Property Description** (150-200 words)
2. **Key Selling Points** in bullet format
 for various platformsions**
4. **Email Marketing Copy** for direct outreach

-maintained condition. It uses active voice and compelling adjectives to create an inviting and persuasive tone.al light, durable hardwood floors, and well

 find the complete marketing materials saved in the file **output/reports/marketing_content.txt**.Marketing content created:
Great! I've created comprehensive marketing content for 123 Maple Street, Suburbia, and saved it to **marketing_content.txt**. The content includes:

1. **Engaging Property Description** (150-200 words)
2. **Key Selling Points** in bullet format
3. **Social Media Captions** for various platforms
4. **Email Marketing Copy** for dire

### Step 4: Market Valuation Analysis

In [13]:
# Perform market valuation
valuation_prompt = f"""
Provide market valuation for: {property_info['address']}
Property details: {json.dumps(property_info)}
Property analysis: {analysis_result}

Get market data for the location and provide comprehensive valuation report.
Save as 'valuation_report.txt'
"""

print("Performing market valuation...")
valuation_result = valuation_agent(valuation_prompt)
print("Valuation completed:")
print(valuation_result)

Performing market valuation...

Tool #1: get_market_data
## Market Valuation Report for 123 Maple Street, Suburbia

### Property Overview
123 Maple Street, Suburbia
**Property Type:** Single Family Home
**Year Built:** 1995
**Lot Size:** 0.25 acres
**Provided Details:**
: 3edrooms
- Bathrooms: 2
- Square Footage: 1,800 sqft

### Market Data Analysis

Based on current market data for Suburbia:

 Foot:** $250-350er Square
**Market Trend:** Rising 5% YoY
**Days on Market:** 25-35 days
**Comparable Sales:** 3 recent sales in area

### Valuation Analysis

#### Conservative Valuation
- Based on lower end of price per square foot: $250/sqft
- Calculation: 1,800 sqft × $250 = $450,000

#### Optimistic Valuation
- Based on higher end of price per square foot: $350/sqft
- Calculation: 1,800 sqft × $350 = $630,000

### Estimated Market Value Range
**$450,000 - $630,000**

### Pricing Strategy Recommendations

 position the property competitively while allowing for negotiation
 a premium within th

### Step 5: Generate Enhanced Marketing Images

In [14]:
# Generate enhanced marketing images
enhancement_prompt = f"""
Create enhanced marketing images for: {property_info['address']}
Based on analysis: {analysis_result}

Generate 3 professional marketing images:
1. Exterior curb appeal shot
2. Spacious interior living area
3. Modern kitchen with natural lighting

Professional real estate photography style with optimal staging.
"""

print("Generating enhanced marketing images...")
enhanced_images = visual_enhancer(enhancement_prompt)
print(f"Enhanced images created: {enhanced_images}")

Generating enhanced marketing images...

Tool #4: generate_image

Tool #5: generate_image

Tool #6: generate_image
The enhanced marketing images for 123 Maple Street, Suburbia have been successfully created, showcasing the property's best features in a professional real estate photography style. These images will help appeal to potential buyers by highlighting the property's curb appeal, spacious interior living area, and modern kitchen with natural lighting. The images adhere to real estate photography best practices, ensuring optimal lighting, staging, and presentation to attract target buyers.  

FINAL DECISION: output/professional_real_estate_exterior_photo.pngEnhanced images created: The enhanced marketing images for 123 Maple Street, Suburbia have been successfully created, showcasing the property's best features in a professional real estate photography style. These images will help appeal to potential buyers by highlighting the property's curb appeal, spacious interior living a

## Complete Property Report Generation

In [ ]:
# Generate comprehensive property report
report_content = f"""
# COMPLETE PROPERTY REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Property Information
{json.dumps(property_info, indent=2)}

## Analysis Summary
{analysis_result}

## Marketing Content
{marketing_result}

## Valuation Report
{valuation_result}

## Generated Images
Original Analysis Images: {sample_images}
Enhanced Marketing Images: {enhanced_images}

---
Report generated by Smart Real Estate Assistant powered by Amazon Nova
"""

# Save complete report
os.makedirs("output/reports", exist_ok=True)
report_filename = f"complete_property_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(f"output/reports/{report_filename}", 'w') as f:
    f.write(report_content)

print(f"Complete property report saved: output/reports/{report_filename}")
print("\n" + "="*50)
print("WORKSHOP COMPLETE!")
print("="*50)

## Key Takeaways

This workshop demonstrated:

1. **Multi-Agent Collaboration**: Four specialized agents working together
2. **Multimodal Processing**: Image analysis, generation, and text processing
3. **Real-World Application**: Practical real estate use case
4. **Tool Integration**: Custom tools for file operations and data retrieval
5. **Workflow Orchestration**: Sequential agent execution with data passing

### Business Value:
- **Time Savings**: Automated analysis and content creation
- **Consistency**: Standardized property evaluation process
- **Enhanced Marketing**: Professional content and visuals
- **Data-Driven Insights**: Market analysis and valuation support

### Next Steps:
- Integrate with MLS APIs for real market data
- Add customer preference matching
- Implement virtual staging capabilities
- Create automated listing distribution